# Lesson 11  - Matrix Multiplication 

 
## Recognizing students

As before,  the video begins with a review of some student work, including some cool improvements to how the guidance parameter is determined. In one case, renomalizing hte parameter seemed to help the model produce more detailed images, and in another case a similar result was obtained by reducing the guidance (using a cosine function) as the denoising proceeded, so that the last few steps had no guidance at all.

## How to read a research paper

THe next section discussed a recent (as of the video date) paper: [DiffEdit: Diffusion-based semantic image editing with mask guidance](https://arxiv.org/abs/2210.11427).   Jeremy used this as an oppurtunity to give some tips on 'how to read a research paper'. I skimmed most of this as it was quite elementary.  

The paper itself discussed a cool way to provide textually guided edits to an image.  The novelty here was automatically generating a mask.  Once the mask was produced, the edit is created by using standard text guided diffusion, except at each step the masked off areas are replaced with an appropriately noised version of the original image. Pretty cool. 

## From the Foundations 

### Matrix multiplication 

[course notebooks](https://github.com/fastai/course22p2)

As you recall we have now basic tensors and random numbers. Now we need to learn to multiply them together. (Matrix multiplication).  Much of this section is also elementary so I will just give broad strokes for those parts.

* He does matrix multiplication as a triple for loop. In python it is very slow

* He then speeds it up with numba. 

* This allows us to use torch matrix multiplication.
 

In [19]:
import numpy as np
import numba as nb
import torch
from minai import mnist_load
from pathlib import Path

path_data = Path('data')
x_train, y_train, x_valid, y_valid = mnist_load.load_data(path_data)
x_train,y_train,x_valid,y_valid = map(torch.tensor, (x_train,y_train,x_valid,y_valid))

In [20]:

weights = torch.randn(784,10)
biases = torch.randn(10)
m1 = x_valid[:10]
m2 = weights

In [4]:

@nb.njit
def dot(a,b):
    res = 0.
    for i in range(len(a)): res+=a[i]*b[i]
    return res
         

test compiling works

In [5]:
%time dot(np.array([1,2,3]),np.array([4,5,6]))

CPU times: user 239 ms, sys: 27.9 ms, total: 267 ms
Wall time: 343 ms


32.0

In [6]:
%time dot(np.array([1,2,3]),np.array([4,5,6]))

CPU times: user 18 μs, sys: 2 μs, total: 20 μs
Wall time: 21.7 μs


32.0

In [21]:
def matmul(a,b):
    (ar,ac),(br,bc) = a.shape,b.shape
    c =np.zeros((ar, bc))
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): c[i,j] += a[i,k] * b[k,j]
    return c

In [22]:

%time matmul(m1,m2);

CPU times: user 540 ms, sys: 196 μs, total: 540 ms
Wall time: 540 ms


In [23]:
# faster version
@nb.njit
def matmul2(a,b):
    (ar,ac),(br,bc) = a.shape,b.shape
    c =np.zeros((ar, bc))
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): c[i,j] += a[i,k] * b[k,j]
    return c

In [35]:
matmul2(m1.numpy(),m2.numpy());  # force compilation
%timeit -n 50 matmul2(m1.numpy(),m2.numpy());

51.8 μs ± 7.27 μs per loop (mean ± std. dev. of 7 runs, 50 loops each)


This is 10,000 times faster then the naive python version

Jermey does a little excursion into APL again, and then shows how pytorch does elementwise addition and multiplication with ease. 

Then he discusses the Frobenius norm, which is the square root of the sum of the squares of the elements of a matrix. (I.e. the 2-norm of a matrix).

$$
\|A\|_F = \sqrt{\sum_{i,j} A_{i,j}^2} = \|A\|_2
$$

I don't know why he brought this up, he then just moves on after showing how to do it in APL. 


### Matrix multiplication elementwise multiplication

Back to matrix multiplication.  We can use the elementwise multiplication to speed up our matrix multiplication.

In [38]:


def matmul(a,b):
    (ar,ac),(br,bc) = a.shape,b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc): c[i,j] = (a[i,:] * b[:,j]).sum()
    return c

%timeit -n 50 matmul(m1, m2);

723 μs ± 24.8 μs per loop (mean ± std. dev. of 7 runs, 50 loops each)


Faster then the triple loop, but not as fast as the numba compiled version. (I compiled the full triple loop. In the video he only compiled the inner loop).

### Broadcasting

- Discussion of broadcasting. (for more see numpy documentation). 
- Jeremy says this also goes back to APL and even further back to a language called Jorick.
- shows how broadcasting can be understood using `expand_as` in pytorch. In numpy you can do this with `broadcast_to` 
- Unsqueeze or indexing with `None` (np.newaxis) can be used to expand dimensions which is very useful for broadcasting. This just inserts a new dimension of size 1.

In [33]:
c = torch.tensor([1,2,3])
c[None] # because a trailing colon can be omitted

tensor([[1, 2, 3]])

In [36]:
m = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
m + c

tensor([[ 2,  4,  6],
        [ 5,  7,  9],
        [ 8, 10, 12]])

Note that this is the same as m + c[None].  But if we want to replicated accross columns we can explicitly use `None` to broadcast the other way

In [38]:
m + c[:,None]

tensor([[ 2,  3,  4],
        [ 6,  7,  8],
        [10, 11, 12]])

In [39]:
c[None,:] * c[:,None]

tensor([[1, 2, 3],
        [2, 4, 6],
        [3, 6, 9]])

This is the outer product!

Broadcasting rules - see [Numpy documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html)

### Matrix multiplication with broadcasting

Take just one image.  This corresponds to one row of the matrix m1. 

In [46]:
digit = m1[0]
digit.shape, m2.shape

(torch.Size([784]), torch.Size([784, 10]))

Now what we want to do is to elementwise multiply this row times every single column in m2. But wait!  We just showed how to do this with broadcasting! We just need to expand m1 in the right way.

In [49]:
digit[:,None].shape

torch.Size([784, 1])

This will expand to 784 x 10 by duplicating the *row* ten times to match the shape of m2, and then they can be elementwise multiplied

In [51]:
(digit[:,None]*m2).shape

torch.Size([784, 10])

This new matrix now just needs to be summed along the rows to get the corresponding row of the output matrix.   

In [52]:
(digit[:,None]*m2).sum(dim=0)

tensor([ -4.7422,  16.1044, -11.0148, -11.8314,   7.4029,  -6.2199, -23.7206,
          2.0668,  -2.5406,   4.1772])

Putting this together we have, so that we now just have one loop over the rows of m1:

In [53]:
def matmul(a,b):
    (ar,ac),(br,bc) = a.shape,b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
#       c[i,j] = (a[i,:] * b[:,j]).sum()      # previous version
        c[i]   = (a[i,:,None] * b).sum(dim=0) # broadcast version
    return c
%timeit -n 500 _=matmul(m1, m2)

87.3 μs ± 2.34 μs per loop (mean ± std. dev. of 7 runs, 500 loops each)


Thats almost as fast as our numba version, and we are still doing the outer loop in python! We can now do the whole data set:

In [45]:
%time matmul(x_train, weights) + biases;

CPU times: user 699 ms, sys: 18.3 ms, total: 717 ms
Wall time: 433 ms


### Torch matmul (Extra, not in the video, will be in the next lesson)
Matrix multiplication is easy in pytorch.  You can use the `@` operator or the `matmul` function. And it is faster  (10x) even then our numba version!

In [54]:
%timeit -n 500  _ = m1 @ m2

6.23 μs ± 407 ns per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [55]:
%timeit -n 5 _ = x_train @ weights + biases

41.3 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


We can do even better using the cuda version, but we will get to that in the next lesson